##Importer les Packages Python Pertinents





In [ ]:
#
import pandas as pd  # Bibliothèques pour la manipulation de données
import numpy as np   # Bibliothèques pour les opérations numériques
import matplotlib.pyplot as plt  # Bibliothèques pour la création de graphiques statiques
import seaborn as sns  # Bibliothèques pour la création de graphiques statistiques informatifs


import plotly.express as px  # Bibliothèques pour la création de graphiques interactifs de type exploratoire
# import plotly.offline as pyoff  # Commentaire pour l'utilisation hors ligne (non utilisé ici)
import plotly.graph_objs as go  # Bibliothèques pour la création de graphiques interactifs complexes
import plotly.figure_factory as ff  # Bibliothèques pour la création de graphiques interactifs spécialisés

# Suppression des messages d'avertissement
import warnings
warnings.filterwarnings("ignore")  # Masquer les messages d'avertissement


import time  # Bibliothèques pour mesurer le temps d'exécution


In [ ]:

# Importation des bibliothèques d'apprentissage automatique
from sklearn.svm import SVC  # Machine à Vecteurs de Support
from sklearn.multioutput import MultiOutputClassifier  # Classifieur multi-sortie
from sklearn.ensemble import GradientBoostingClassifier  # Classifieur à Gradient Boosting
from sklearn.tree import DecisionTreeClassifier  # Classifieur par arbre de décision
from sklearn.neighbors import KNeighborsClassifier  # Classifieur K plus proches voisins
from sklearn.naive_bayes import GaussianNB  # Classifieur naïf bayésien gaussien
from sklearn.ensemble import RandomForestClassifier  # Classifieur Random Forest
from sklearn.linear_model import LogisticRegression  # Régression logistique
from sklearn.model_selection import (  # Bibliothèques pour la sélection de modèles
    KFold,  # Validation croisée K-fold
    cross_val_score,  # Calcul des scores de validation croisée
    train_test_split,  # Séparation des données en ensembles d'entraînement et de test
    GridSearchCV,  # Recherche de paramètres par grille
    cross_validate  # Validation croisée généralisée
)

In [ ]:

from multiscorer import MultiScorer  # Création d'un scoreur multi-critère
from sklearn.metrics import (  # Bibliothèques d'évaluation des modèles
    accuracy_score,  # Taux de l'exactitude
    f1_score,  # Score F1
    precision_score,  # Précision
    recall_score,  # Rappel
    confusion_matrix  # Matrice de confusion
)
from sklearn.cluster import KMeans  # Bibliothèques pour le clustering
import xgboost as xgb  # XGBoost (bibliothèque d'arbres à gradient optimisés)
df=None

##importation du data principale

In [ ]:

# Loading the data
df = pd.read_csv('DT.csv')
df = df.drop(df.columns[0], axis=1)
df.head(4)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,UTRO 200MG 2X14C PNO,12,2021-12-01 07:45:00,6.95,13085.0,France
1,489434,79323P,OESTROGEL 80G,12,2021-12-01 07:45:00,6.75,13085.0,France
2,489434,79323W,UTRO 200MG 14CAPS SFO,12,2021-12-01 07:45:00,6.75,13085.0,France
3,489434,22041,OGESTAN 144MG 30CAPS,48,2021-12-01 07:45:00,2.10,13085.0,France


## ingenierie de donnée

Dans cette section, nous allons explorer l'ensemble de données donné pour répondre à certaines questions générales sur l'ensemble de données. Cela nous amènera à introduire d'autres caractéristiques dans l'ensemble de données qui nous aideront à construire un modèle d'apprentissage automatique qui nous aidera à répondre à notre question principale.

Mettons à jour le dataframe pandas df en renommant certaines de ses colonnes.

In [ ]:


df.rename(columns={'Invoice':'FactureNo', 'Customer ID':'ClientID', 'Price':'PrixUnitaire','Quantity':'Quantité',
                   'Pays':'Pays','FactureDate':'DateFacture','InvoiceDate':'FactureDate','Country':'Pays'}, inplace=True)
df.head()

,FactureNo,StockCode,Description,Quantité,FactureDate,PrixUnitaire,ClientID,Pays
0,489434,85048,UTRO 200MG 2X14C PNO,12,2021-12-01 07:45:00,6.95,13085.0,France
1,489434,79323P,OESTROGEL 80G,12,2021-12-01 07:45:00,6.75,13085.0,France
2,489434,79323W,UTRO 200MG 14CAPS SFO,12,2021-12-01 07:45:00,6.75,13085.0,France
3,489434,22041,OGESTAN 144MG 30CAPS,48,2021-12-01 07:45:00,2.10,13085.0,France
4,489434,21232,OGESTAN PLUS 30CAPS,24,2021-12-01 07:45:00,1.25,13085.0,France


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053041 entries, 0 to 1053040
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   FactureNo     1053041 non-null  object 
 1   StockCode     1053041 non-null  object 
 2   Description   1053041 non-null  object 
 3   Quantité      1053041 non-null  int64  
 4   FactureDate   1053041 non-null  object 
 5   PrixUnitaire  1053041 non-null  float64
 6   ClientID      810162 non-null   float64
 7   Pays          1053041 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 64.3+ MB


In [ ]:
df.isnull().sum()

FactureNo            0
StockCode            0
Description          0
Quantité             0
FactureDate          0
PrixUnitaire         0
ClientID        242879
Pays                 0
dtype: int64

les colonnes « ClientID » contient  242879 données manquantes. Les données manquantes dans la colonne « ClientID » représentent 23 % de l'ensemble des clients en ligne.

In [ ]:
np.round(df.isnull().sum()["ClientID"]/df.FactureNo.shape[0] * 100, 2)

23.06

Comment traitons-nous ces données manquantes ? Après avoir discuté avec les chefs d'entreprise, ils ont suggéré de supprimer l'élément dont le ClientId est manquant. On peut y parvenir grâce à la méthode dropna.

In [ ]:
df_data = df.dropna()
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 810162 entries, 0 to 1053040
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   FactureNo     810162 non-null  object 
 1   StockCode     810162 non-null  object 
 2   Description   810162 non-null  object 
 3   Quantité      810162 non-null  int64  
 4   FactureDate   810162 non-null  object 
 5   PrixUnitaire  810162 non-null  float64
 6   ClientID      810162 non-null  float64
 7   Pays          810162 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 55.6+ MB


Ensuite, nous mettons à jour le dataframe df_data en convertissant le champ de date, FactureDate, en objet de type datetime. On peut utiliser la méthode to_datetime pour y parvenir.

In [ ]:
df_data.FactureDate = pd.to_datetime(df_data.FactureDate)
df_data.head()

,FactureNo,StockCode,Description,Quantité,FactureDate,PrixUnitaire,ClientID,Pays
0,489434,85048,UTRO 200MG 2X14C PNO,12,2021-12-01 07:45:00,6.95,13085.0,France
1,489434,79323P,OESTROGEL 80G,12,2021-12-01 07:45:00,6.75,13085.0,France
2,489434,79323W,UTRO 200MG 14CAPS SFO,12,2021-12-01 07:45:00,6.75,13085.0,France
3,489434,22041,OGESTAN 144MG 30CAPS,48,2021-12-01 07:45:00,2.10,13085.0,France
4,489434,21232,OGESTAN PLUS 30CAPS,24,2021-12-01 07:45:00,1.25,13085.0,France


In [ ]:
df['FactureDate'] = pd.to_datetime(df['FactureDate'])


In [ ]:
pd.DataFrame(df_data.FactureDate.describe())

,FactureDate
count,810162
mean,2023-01-01 04:18:22.338051584
min,2021-12-01 07:45:00
25%,2022-07-05 13:14:00
50%,2022-12-02 19:18:00
75%,2023-07-26 15:06:30
max,2023-12-09 12:49:00


D'après ce qui précède, nous constatons que les achats en ligne effectués par les clients se situent entre le 01/12/2021 et le 09/12/2023.

Dans les cellules suivantes, nous allons répondre à quelques questions que nous aimerions poser à partir de l'ensemble de données fourni.


Combien de clients en ligne y a-t-il dans l'ensemble de données et quel est leur pays d'origine ?

In [ ]:
print('D\'après l\'ensemble de données, le magasin de vente en ligne compte **{} clients** de **{} pays** différents.'.format(len(df_data.ClientID.unique()), len(df_data.Pays.unique())))

D'après l'ensemble de données, le magasin de vente en ligne compte **5849 clients** de **40 pays** différents.


## Question 2

Quels sont les pays les plus représentés dans l'ensemble de données ?


In [ ]:
cust_cntry_df = df_data.groupby(['ClientID', 'Pays']).count().reset_index()

cust_cntry_df = cust_cntry_df.groupby('Pays')['ClientID'].count().reset_index().sort_values(
    by=['ClientID'], ascending=False)

# Create a new column, Percentage to calculate the customer representation in percentage
cust_cntry_df['Percentage']= np.round(cust_cntry_df.ClientID / cust_cntry_df.ClientID.sum() * 100, 2)

cust_cntry_df.head(10)

,Pays,ClientID,Percentage
13,France,5410,92.32
14,Germany,107,1.83
32,Spain,41,0.70
3,Belgium,29,0.49
28,Portugal,24,0.41
24,Netherlands,23,0.39
34,Switzerland,22,0.38
33,Sweden,19,0.32
18,Italy,17,0.29
12,Finland,15,0.26


Les résultats ci-dessus montrent les 10 principaux pays qui effectuent des achats dans la boutique en ligne. La France arrive en tête de liste avec le nombre de clients le plus élevé. Ce pays représentent environ 92 % de l'ensemble des clients en ligne.

Bien que 13 clients en ligne aient indiqué qu'ils vivaient dans deux pays, cela n'affecte pas significativement les pourcentages enregistrés. Pour ces *13* clients, il est possible que les pays enregistrés pour leurs derniers achats correspondent à leur lieu de résidence actuel.


In [ ]:
#Initialisation d'un dictionnaire. Celui-ci est destiné à collecter tous les clients
#ayant indiqué plus d'un pays.
cust_2_cntry = {}

# Pour chaque index et identifiant client
for idx, cid in enumerate(df_data.ClientID.unique()):

    # Obtenir le pays de résidence du client
    cntry = df_data[df_data.ClientID == cid].Pays.unique()

    # Si le client a vécu dans plus d'un pays
    if len(cntry) > 1:

        # Mettre à jour le dictionnaire 'cust_2_cntry'
        cust_2_cntry[cid] = cntry

pd.DataFrame(cust_2_cntry)

,12417.0,12455.0,12422.0,12431.0,12745.0,12370.0,12429.0,12423.0,12449.0,12457.0,12394.0
0,Belgium,Cyprus,Australia,Belgium,EIRE,Austria,Denmark,Denmark,Denmark,Switzerland,Belgium
1,Spain,Spain,Switzerland,Australia,France,Cyprus,Austria,Belgium,Belgium,Cyprus,Denmark


Dans les deux cellules suivantes, nous allons regrouper les pays dont le pourcentage de clients est inférieur ou égal à 0,25 et ensuite tracer un diagramme circulaire .


In [ ]:
percent_margin = 0.25

#Créer une nouvelle colonne, PaysCategory et définir les valeurs
#sur les valeurs correspondantes de la colonne Pays
cust_cntry_df['PaysCategory'] = cust_cntry_df.Pays

#Définir les pays avec un pourcentage
#inférieur ou égal à percent_margin à 'Pays '

cust_cntry_df.loc[cust_cntry_df.Percentage <= percent_margin, 'PaysCategory'] = 'Autres Pays'

cust_cntry_df.head(11)

,Pays,ClientID,Percentage,PaysCategory
13,France,5410,92.32,France
14,Germany,107,1.83,Germany
32,Spain,41,0.70,Spain
3,Belgium,29,0.49,Belgium
28,Portugal,24,0.41,Portugal
24,Netherlands,23,0.39,Netherlands
34,Switzerland,22,0.38,Switzerland
33,Sweden,19,0.32,Sweden
18,Italy,17,0.29,Italy
12,Finland,15,0.26,Finland


In [ ]:

# diagramme circulaire

pie_fig = px.pie(cust_cntry_df,
                 names="PaysCategory",
                 values="Percentage",
                 title="",
                 template= "plotly_dark"
                )

pie_fig.update_layout(title_x=0,
                      legend_title="Les Pays",
                      legend=dict(orientation="h")
                     )

pie_fig.show(config={'displaylogo': False})

## Question 3

Calculer le chiffre d'affaires réalisé chaque mois et quel est le pourcentage de chiffre d'affaires par pays ?

In [ ]:
df_data['InvoiceYearMonth'] = df_data['FactureDate'].map(lambda date: 100*date.year + date.month)
df_data.head()

,FactureNo,StockCode,Description,Quantité,FactureDate,PrixUnitaire,ClientID,Pays,InvoiceYearMonth
0,489434,85048,UTRO 200MG 2X14C PNO,12,2021-12-01 07:45:00,6.95,13085.0,France,202112
1,489434,79323P,OESTROGEL 80G,12,2021-12-01 07:45:00,6.75,13085.0,France,202112
2,489434,79323W,UTRO 200MG 14CAPS SFO,12,2021-12-01 07:45:00,6.75,13085.0,France,202112
3,489434,22041,OGESTAN 144MG 30CAPS,48,2021-12-01 07:45:00,2.10,13085.0,France,202112
4,489434,21232,OGESTAN PLUS 30CAPS,24,2021-12-01 07:45:00,1.25,13085.0,France,202112


In [ ]:
df_data['Revenue'] = df_data.PrixUnitaire * df_data.Quantité
df_data.head()

,FactureNo,StockCode,Description,Quantité,FactureDate,PrixUnitaire,ClientID,Pays,InvoiceYearMonth,Revenue
0,489434,85048,UTRO 200MG 2X14C PNO,12,2021-12-01 07:45:00,6.95,13085.0,France,202112,83.4
1,489434,79323P,OESTROGEL 80G,12,2021-12-01 07:45:00,6.75,13085.0,France,202112,81.0
2,489434,79323W,UTRO 200MG 14CAPS SFO,12,2021-12-01 07:45:00,6.75,13085.0,France,202112,81.0
3,489434,22041,OGESTAN 144MG 30CAPS,48,2021-12-01 07:45:00,2.10,13085.0,France,202112,100.8
4,489434,21232,OGESTAN PLUS 30CAPS,24,2021-12-01 07:45:00,1.25,13085.0,France,202112,30.0


In [ ]:
cust_revenue = df_data.groupby('InvoiceYearMonth').Revenue.sum().reset_index()
cust_revenue.head(6)

,InvoiceYearMonth,Revenue
0,202112,657717.010
1,202201,523603.892
2,202202,480418.576
3,202203,627565.261
4,202204,553269.912
5,202205,546478.050


In [ ]:
pd.DataFrame(cust_revenue.Revenue.describe())

,Revenue
count,2.500000e+01
mean,6.528715e+05
std,2.110428e+05
min,3.354575e+05
25%,5.464781e+05
50%,5.711075e+05
75%,7.797085e+05
max,1.116001e+06


In [ ]:
#
line_fig = px.line(cust_revenue,
                   x = "InvoiceYearMonth",
                   y = "Revenue",
                   title = "Revenue monsuele de Dec. 2021 vers Dec. 2023",
                   template= "plotly_dark"
                  )

line_fig.update_layout(title_x=0.5,
                       showlegend=False,
                       xaxis={"type": "category"},
                       xaxis_title="Facture Année-Mois",
                       yaxis_title="Revenue monsuele"
                      )

line_fig.show(config={'displaylogo': False})

In [ ]:
cntry_revenue_df = df_data.groupby(['Pays']).Revenue.sum().reset_index().sort_values(by=['Revenue'],
                                                                                        ascending=False)

cntry_revenue_df['Percentage'] = np.round(cntry_revenue_df.Revenue / cntry_revenue_df.Revenue.sum() * 100, 2)

cntry_revenue_df.head(5)

,Pays,Revenue,Percentage
13,France,1.380642e+07,84.59
10,EIRE,5.785016e+05,3.54
24,Netherlands,5.485249e+05,3.36
14,Germany,4.179886e+05,2.56
0,Australia,1.671291e+05,1.02


D'après les résultats ci-dessus, les 5 principaux pays en termes de chiffre d'affaires généré sont :

*La France

*La République d'Irlande (EIRE)

*Netherlands

*Allemagne

*Australli

avec la France enregistrant le plus haut pourcentage (84,59%).

Dans les deux prochaines cellules, nous mettons à jour cntry_revenue_df avec un nouveau

Dans les deux prochaines cellules, nous regroupons les pays avec un pourcentage de chiffre d'affaires inférieur ou égal à 0,25 et traçons ensuite un graphique circulaire.



In [ ]:
percent_margin = 0.25

# Créer une nouvelle colonne, PaysCategory, et définir ses valeurs en fonction de la colonne Pays.
cntry_revenue_df['PaysCategory'] = cntry_revenue_df.Pays

# Définir les pays dont le pourcentage est inférieur ou égal à percent_margin comme "Autres pays".

cntry_revenue_df.loc[cntry_revenue_df.Percentage <= percent_margin, 'PaysCategory'] = 'Autres Pays'

cntry_revenue_df.head(11)

,Pays,Revenue,Percentage,PaysCategory
13,France,1.380642e+07,84.59,France
10,EIRE,5.785016e+05,3.54,EIRE
24,Netherlands,5.485249e+05,3.36,Netherlands
14,Germany,4.179886e+05,2.56,Germany
0,Australia,1.671291e+05,1.02,Australia
34,Switzerland,9.908281e+04,0.61,Switzerland
32,Spain,9.185948e+04,0.56,Spain
33,Sweden,8.745542e+04,0.54,Sweden
9,Denmark,6.574109e+04,0.40,Denmark
3,Belgium,6.357449e+04,0.39,Belgium


In [ ]:
# plot pie chart
pie_fig = px.pie(cntry_revenue_df,
                 names="PaysCategory",
                 values="Percentage",
                 title="Percentage des revenue des Pays",
                 template= "plotly_dark"
                )

pie_fig.update_layout(title_x=0,
                      legend_title="Pays",
                      legend=dict(orientation="h")
                     )

pie_fig.show(config={'displaylogo': False})

## Prédiction des achats des clients

L'objectif de cette section est de créer un modèle en utilisant le dataframe donné df_data pour estimer si un client donné effectuera un nouvel achat du prochain trimestre.

Le dataframe est divisé en deux parties.

La première sous-partie de données, assignée à la variable  cust_bhvr_dt, contient les achats effectués par les clients du 01-12-2021 au 30-08-2023. Ce dataframe sera utilisé pour étudier les comportements d'achat des clients en ligne.

La deuxième sous-partie de données, assignée à la variable Python cust_next_quarter, sera utilisée pour étudier les comportements d'achat des clients au cours du prochain trimestre. Cela correspond à la période du 01-09-2023 au 30-11-2023.

In [ ]:
df_data.head()

,FactureNo,StockCode,Description,Quantité,FactureDate,PrixUnitaire,ClientID,Pays,InvoiceYearMonth,Revenue
0,489434,85048,UTRO 200MG 2X14C PNO,12,2021-12-01 07:45:00,6.95,13085.0,France,202112,83.4
1,489434,79323P,OESTROGEL 80G,12,2021-12-01 07:45:00,6.75,13085.0,France,202112,81.0
2,489434,79323W,UTRO 200MG 14CAPS SFO,12,2021-12-01 07:45:00,6.75,13085.0,France,202112,81.0
3,489434,22041,OGESTAN 144MG 30CAPS,48,2021-12-01 07:45:00,2.10,13085.0,France,202112,100.8
4,489434,21232,OGESTAN PLUS 30CAPS,24,2021-12-01 07:45:00,1.25,13085.0,France,202112,30.0


In [ ]:
cust_bhvr_dt = df_data[(df_data.FactureDate < pd.Timestamp(2023,9,1)) &
      (df_data.FactureDate >= pd.Timestamp(2021,12,1))].reset_index(drop=True)


cust_next_quarter = df_data[(df_data.FactureDate < pd.Timestamp(2023,12,1)) &
      (df_data.FactureDate >= pd.Timestamp(2023,9,1))].reset_index(drop=True)

In [ ]:
0# Obtenir les clients distincts dans le dataframe cust_bhvr_dt
cust_data = pd.DataFrame(cust_bhvr_dt['ClientID'].unique())
# Renommer la colonne en ClientID.
cust_data.columns = ['ClientID']

cust_data.head()

,ClientID
0,13085.0
1,13078.0
2,15362.0
3,18102.0
4,18087.0


In [ ]:
#Créer un dataframe avec ClientID et la première date d'achat des clients dans cust_next_quarter
cust_1st_purchase_in_next_quarter = cust_next_quarter.groupby('ClientID').FactureDate.min().reset_index()
cust_1st_purchase_in_next_quarter.columns = ['ClientID','MinPurchaseDate']
cust_1st_purchase_in_next_quarter.head()

,ClientID,MinPurchaseDate
0,12347.0,2023-10-31 12:25:00
1,12348.0,2023-09-25 13:13:00
2,12349.0,2023-11-21 09:51:00
3,12352.0,2023-09-20 14:34:00
4,12356.0,2023-11-17 08:40:00


In [ ]:
##Créer un dataframe avec ClientID et la derniére date d'achat des clients dans cust_next_quarter
cust_last_purchase_bhvr_dt = cust_bhvr_dt.groupby('ClientID').FactureDate.max().reset_index()
cust_last_purchase_bhvr_dt.columns = ['ClientID','MaxPurchaseDate']
cust_last_purchase_bhvr_dt.head()

,ClientID,MaxPurchaseDate
0,12346.0,2023-01-18 10:17:00
1,12347.0,2023-08-02 08:48:00
2,12348.0,2023-04-05 10:47:00
3,12349.0,2022-10-28 08:23:00
4,12350.0,2023-02-02 16:01:00


In [ ]:
#Fusionner deux dataframes, cust_last_purchase_bhvr_dt et cust_1st_purchase_in_next_quarter
cust_purchase_dates = pd.merge(cust_last_purchase_bhvr_dt, cust_1st_purchase_in_next_quarter, on='ClientID',
                              how='left')
cust_purchase_dates.head()

,ClientID,MaxPurchaseDate,MinPurchaseDate
0,12346.0,2023-01-18 10:17:00,NaT
1,12347.0,2023-08-02 08:48:00,2023-10-31 12:25:00
2,12348.0,2023-04-05 10:47:00,2023-09-25 13:13:00
3,12349.0,2022-10-28 08:23:00,2023-11-21 09:51:00
4,12350.0,2023-02-02 16:01:00,NaT


Calculons la différence de temps en jours entre le dernier achat du client dans le dataframe cust_last_purchase_bhvr_dt et le premier
achat dans le dataframe cust_1st_purchase_in_next_quarter.

In [ ]:
cust_purchase_dates['cust'] = (cust_purchase_dates['MinPurchaseDate'] - cust_purchase_dates['MaxPurchaseDate']).dt.days

cust_purchase_dates.head()

,ClientID,MaxPurchaseDate,MinPurchaseDate,cust
0,12346.0,2023-01-18 10:17:00,NaT,NaN
1,12347.0,2023-08-02 08:48:00,2023-10-31 12:25:00,90.0
2,12348.0,2023-04-05 10:47:00,2023-09-25 13:13:00,173.0
3,12349.0,2022-10-28 08:23:00,2023-11-21 09:51:00,389.0
4,12350.0,2023-02-02 16:01:00,NaT,NaN


In [ ]:
# fusionner avec cust_data
cust_data = pd.merge(cust_data, cust_purchase_dates[['ClientID','cust']], on='ClientID', how='left')
cust_data.head()

,ClientID,cust
0,13085.0,NaN
1,13078.0,13.0
2,15362.0,NaN
3,18102.0,27.0
4,18087.0,46.0


Mettez à jour le dataframe cust_data en remplissant toutes les valeurs manquantes avec 9999.

In [ ]:
cust_data = cust_data.fillna(9999)
cust_data.head()

,ClientID,cust
0,13085.0,9999.0
1,13078.0,13.0
2,15362.0,9999.0
3,18102.0,27.0
4,18087.0,46.0


Ensuite, nous allons définir certaines caractéristiques et les ajouter au dataframe cust_data pour construire notre modèle d'apprentissage automatique. Nous utiliserons la méthode de segmentation Récence - Fréquence - Valeur Monétaire. Cela signifie que nous placerons les clients dans des groupes en fonction des critères suivants :

Récence : Le comportement d'achat des clients en fonction de leur date d'achat la plus récente et du nombre de jours écoulés depuis leur dernier achat.

Fréquence : Le comportement d'achat des clients en fonction du nombre de fois où ils achètent sur la boutique en ligne.

Valeur Monétaire/Revenu : Le comportement d'achat des clients en fonction du chiffre d'affaires qu'ils génèrent.

Ensuite, nous appliquerons la méthode de regroupement K-means pour attribuer à chaque client un score pour chacune de ces caractéristiques.

In [ ]:
cust_max_purchase = cust_bhvr_dt.groupby('ClientID').FactureDate.max().reset_index()
cust_max_purchase.columns = ['ClientID','MaxPurchaseDate']
cust_max_purchase.head()

,ClientID,MaxPurchaseDate
0,12346.0,2023-01-18 10:17:00
1,12347.0,2023-08-02 08:48:00
2,12348.0,2023-04-05 10:47:00
3,12349.0,2022-10-28 08:23:00
4,12350.0,2023-02-02 16:01:00


### Récence

Trouvons la date d'achat la plus récente de chaque client et voyons depuis combien de jours ils sont inactifs. Ensuite, nous pourrons appliquer le regroupement K-means pour attribuer à chaque client un score de récence.

In [ ]:
# Trouver la récence en jours
cust_max_purchase['Récence'] = (cust_max_purchase['MaxPurchaseDate'].max() - cust_max_purchase['MaxPurchaseDate']).dt.days

# Fusionner les dataframes cust_data et cust_max_purchase[['ClientID', 'Recence']]
#sur la colonne ClientID.
cust_data = pd.merge(cust_data, cust_max_purchase[['ClientID', 'Récence']], on='ClientID')
cust_data.head()

,ClientID,cust,Récence
0,13085.0,9999.0,57
1,13078.0,13.0,0
2,15362.0,9999.0,348
3,18102.0,27.0,26
4,18087.0,46.0,44


In [ ]:
cust_max_purchase['MaxPurchaseDate'].max()

Timestamp('2023-08-31 17:45:00')

In [ ]:
pd.DataFrame(cust_data.Récence.describe())



,Récence
count,5239.000000
mean,205.876885
std,173.819485
min,0.000000
25%,48.000000
50%,162.000000
75%,320.500000
max,638.000000


La récence moyenne est d'environ 205 jours tandis que la médiane est de 162 jours.

In [ ]:
# Histogramme
hist_fig = px.histogram(cust_data,
                        x="Récence",
                        title="récence des client par jours",
                        template= "plotly_dark"
                       )

hist_fig.update_layout(title_x=0.5,
                       xaxis_title="La récence est regroupée par intervalles de 20 jours.",
                       yaxis_title="Nombre de client"
                      )

hist_fig.show(config={'displaylogo': False})

Ensuite, nous appliquerons le regroupement K-means pour attribuer un score de récence. Cependant, nous devons savoir combien de groupes utiliser dans l'algorithme K-means. Nous appliquerons la méthode du coude (Elbow Method) pour déterminer le nombre optimal de groupes dont nous avons besoin. La méthode du coude nous indique simplement le nombre optimal de groupes en fonction de l'inertie optimale.

In [ ]:
my_dict={}
cust_Récence = cust_data[['Récence']]
for idx in range(1, 10):
    kmeans = KMeans(n_clusters=idx, max_iter=1000).fit(cust_Récence)
    cust_Récence["clusters"] = kmeans.labels_
    my_dict[idx] = kmeans.inertia_

line_fig = px.line(x=list(my_dict.keys()),
                   y=list(my_dict.values()),
                   template="plotly_dark"
                  )

line_fig.update_layout(title_x=0,
                       xaxis_title="Nombre de Cluster",
                       yaxis_title=""
                      )

line_fig.show(config={'displaylogo': False})

D'après la figure ci-dessus, il semble que 4 soit le nombre optimal.

In [ ]:
nombre_de_clusters = 4

In [ ]:
# Construisons 4 groupes (clusters) pour la récence et ajoutons-les au dataframe cust_data.

In [ ]:
kmeans = KMeans(n_clusters=nombre_de_clusters)
kmeans.fit(cust_data[['Récence']])
cust_data['RécenceCluster'] = kmeans.predict(cust_data[['Récence']])
cust_data.head()

,ClientID,cust,Récence,RécenceCluster
0,13085.0,9999.0,57,2
1,13078.0,13.0,0,2
2,15362.0,9999.0,348,3
3,18102.0,27.0,26,2
4,18087.0,46.0,44,2


In [ ]:
def order_cluster(df, target_field_name, cluster_field_name, ascending):
  """"
INPUT:
- df - DataFrame pandas
- target_field_name - str - Une colonne dans le DataFrame pandas df
- cluster_field_name - str - Supposé être une colonne dans le DataFrame pandas df
- ascending - Booléen

OUTPUT:
- df_final - DataFrame pandas avec target_field_name et cluster_field_name comme colonnes
  """
# Ajouter la chaîne de caractères "new_" à cluster_field_name
  new_cluster_field_name = "new_" + cluster_field_name

# Créer un nouveau dataframe en regroupant le dataframe d'entrée par cluster_field_name et en extrayant
# target_field_name et en calculant la moyenne
  df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()

# Trier le nouveau dataframe df_new, par target_field_name en ordre décroissant
  df_new = df_new.sort_values(by=target_field_name, ascending=ascending).reset_index(drop=True)

# Créer une nouvelle colonne dans df_new avec le nom de colonne "index" et lui assigner les valeurs df_new.index
  df_new["index"] = df_new.index

# Créer un nouveau dataframe en fusionnant le dataframe d'entrée df et une partie des colonnes de df_new
# basé sur cluster_field_name
  df_final = pd.merge(df, df_new[[cluster_field_name, "index"]], on=cluster_field_name)

# Mettre à jour le dataframe df_final en supprimant la colonne cluster_field_name
  df_final = df_final.drop([cluster_field_name], axis=1)

# Renommer la colonne index en cluster_field_name
  df_final = df_final.rename(columns={"index": cluster_field_name})

  return df_final

In [ ]:
cust_data = order_cluster(cust_data, 'Récence', 'RécenceCluster', False)
cust_data.head()


,ClientID,cust,Récence,RécenceCluster
0,13085.0,9999.0,57,3
1,13078.0,13.0,0,3
2,18102.0,27.0,26,3
3,18087.0,46.0,44,3
4,14110.0,30.0,19,3


Celui-ci permet d'obtenir une vision plus détaillée des caractéristiques de chaque cluster en termes de récence des clients.

In [ ]:
#Afficher les caractéristiques des groupes de récence
cust_data.groupby('RécenceCluster')['Récence'].describe()

,count,mean,std,min,25%,50%,75%,max
RécenceCluster,,,,,,,,
0,746.0,520.624665,65.207721,418.0,465.5,517.0,571.5,638.0
1,1420.0,314.419718,40.961487,236.0,282.0,308.0,338.0,416.0
2,1081.0,155.471785,39.070300,97.0,118.0,153.0,187.0,234.0
3,1992.0,37.982932,27.709237,0.0,13.0,33.0,59.0,96.0


Observons à partir des informations ci-dessus que le groupe 3 regroupe les clients les plus récents tandis que le groupe 0 regroupe les clients les plus inactifs.

### Fréquence

Ensuite, nous examinerons le comportement d'achat des clients en fonction du nombre de fois où ils effectuent des achats sur la boutique de vente en ligne. C'est-à-dire, le nombre total de commandes effectuées par chaque client.

In [ ]:

#obtenir le nombre de commandes pour chaque utilisateur et créer un dataframe avec ces données
cust_Fréquence = df_data.groupby('ClientID').FactureDate.count().reset_index()
cust_Fréquence.columns = ['ClientID','Fréquence']

In [ ]:
#ajouter ces données à notre dataframe principal cust_data
cust_data = pd.merge(cust_data, cust_Fréquence, on='ClientID')

cust_data.head()

,ClientID,cust,Récence,RécenceCluster,Fréquence
0,13085.0,9999.0,57,3,92
1,13078.0,13.0,0,3,855
2,18102.0,27.0,26,3,1068
3,18087.0,46.0,44,3,95
4,14110.0,30.0,19,3,403


In [ ]:
pd.DataFrame(cust_data.Fréquence.describe())

,Fréquence
count,5239.000000
mean,149.115862
std,379.652064
min,1.000000
25%,22.000000
50%,59.000000
75%,158.000000
max,13097.000000


In [ ]:
cust_data.groupby('RécenceCluster')['Fréquence'].describe()

,count,mean,std,min,25%,50%,75%,max
RécenceCluster,,,,,,,,
0,746.0,28.324397,36.095427,1.0,7.0,18.0,37.0,367.0
1,1420.0,70.611972,84.263510,1.0,19.0,42.5,88.0,707.0
2,1081.0,109.145236,159.012282,1.0,24.0,58.0,136.0,2827.0
3,1992.0,272.004518,578.009753,1.0,56.0,137.0,302.0,13097.0


Méme le cas de récence si le nombre du cluster augmente ca veut un bon client ,car il a un grand moyen

In [ ]:
#histogramme
hist_fig = px.histogram(x=cust_data.query('Fréquence < 1200')['Fréquence'],
                        title="fréquence d'achat des client inférieur a 1200"
                        ,
                        template= "plotly_dark"
                       )

hist_fig.update_layout(title_x=0.5,
                       xaxis_title="frequence d\'achat des client par group de 20",
                       yaxis_title="nombre des client"
                      )

hist_fig.show(config={'displaylogo': False})

In [ ]:
kmeans = KMeans(n_clusters=nombre_de_clusters)
kmeans.fit(cust_data[['Fréquence']])
cust_data['FréquenceCluster'] = kmeans.predict(cust_data[['Fréquence']])

In [ ]:
cust_data = order_cluster(cust_data, 'Fréquence', 'FréquenceCluster', False)
cust_data.head()

,ClientID,cust,Récence,RécenceCluster,Fréquence,FréquenceCluster
0,13085.0,9999.0,57,3,92,3
1,18087.0,46.0,44,3,95,3
2,17519.0,116.0,33,3,224,3
3,12362.0,40.0,12,3,275,3
4,15712.0,38.0,9,3,167,3


In [ ]:
#Pour voir les detaills de chaque cluster
cust_data.groupby('FréquenceCluster')['Fréquence'].describe()

,count,mean,std,min,25%,50%,75%,max
FréquenceCluster,,,,,,,,
0,2.0,12355.000000,1049.346463,11613.0,11984.00,12355.0,12726.00,13097.0
1,14.0,3921.000000,1541.056680,2430.0,2829.50,3346.5,4570.25,7307.0
2,394.0,716.967005,319.033857,403.0,483.25,604.0,849.50,2134.0
3,4829.0,86.794160,91.088105,1.0,20.00,51.0,123.00,402.0



le nombre elevé de fréquence veut dire des meilleurs clients

In [ ]:
cust_revenue = df_data.groupby('ClientID').Revenue.sum().reset_index()

In [ ]:
#fusionner avec cust_data
cust_data = pd.merge(cust_data, cust_revenue, on='ClientID')
cust_data.head()

,ClientID,cust,Récence,RécenceCluster,Fréquence,FréquenceCluster,Revenue
0,13085.0,9999.0,57,3,92,3,1459.46
1,18087.0,46.0,44,3,95,3,14411.62
2,17519.0,116.0,33,3,224,3,5102.80
3,12362.0,40.0,12,3,275,3,5284.58
4,15712.0,38.0,9,3,167,3,3467.46


In [ ]:
# histogramme
hist_fig = px.histogram(x=cust_data.query('Revenue < 10000')['Revenue'],
                        title="Clients ayant une valeur monétaire inférieure à 10000."
                        ,
                        template= "plotly_dark"
                       )

hist_fig.update_layout(title_x=0.5,
                       xaxis_title="Client Revenue",
                       yaxis_title="Nombre de Client"
                      )

hist_fig.show(config={'displaylogo': False})

In [ ]:
#appliquons le  clustering sur le revenue
kmeans = KMeans(n_clusters=nombre_de_clusters)
kmeans.fit(cust_data[['Revenue']])
cust_data['RevenueCluster'] = kmeans.predict(cust_data[['Revenue']])

In [ ]:
#ordonner les nombre de cluster
cust_data = order_cluster(cust_data, 'Revenue', 'RevenueCluster', True)
cust_data.head()

,ClientID,cust,Récence,RécenceCluster,Fréquence,FréquenceCluster,Revenue,RevenueCluster
0,13085.0,9999.0,57,3,92,3,1459.46,0
1,18087.0,46.0,44,3,95,3,14411.62,0
2,17519.0,116.0,33,3,224,3,5102.80,0
3,12362.0,40.0,12,3,275,3,5284.58,0
4,15712.0,38.0,9,3,167,3,3467.46,0


In [ ]:
#afficher les détailles du dataframes
cust_data.groupby('RevenueCluster')['Revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
RevenueCluster,,,,,,,,
0,5183.0,2086.257854,3319.296375,-25111.09,343.9100,925.770,2348.7350,26960.02
1,46.0,52344.501804,22655.252989,27264.45,34758.5075,47988.475,60414.9425,113513.07
2,8.0,198033.391250,62010.602305,136391.48,142827.5300,181355.750,242746.6750,296564.69
3,2.0,560778.645000,52943.312094,523342.07,542060.3575,560778.645,579496.9325,598215.22


### Score global.

In [ ]:
# Calculer le score global et utiliser la fonction moyenne() pour voir les détails
cust_data['ScoreGlobale'] = cust_data['RécenceCluster'] + cust_data['FréquenceCluster'] + cust_data['RevenueCluster']

# Grouper par 'ScoreGlobale' et calculer la moyenne des colonnes 'Récence', 'Fréquence' et 'Revenue'
cust_data.groupby('ScoreGlobale')[['Récence', 'Fréquence', 'Revenue']].mean()

,Récence,Fréquence,Revenue
ScoreGlobale,,,
3,514.695026,40.060209,436.253353
4,309.086207,96.647586,1235.875396
5,126.341753,265.257058,3884.646324
6,40.269115,144.069235,3533.487231
7,12.823529,768.823529,123844.565294
8,26.000000,1068.000000,598215.220000


In [ ]:
cust_data['Segment'] = 'Low-Value'
cust_data.loc[cust_data['ScoreGlobale'] > 4, 'Segment'] = 'Mid-Value'
cust_data.loc[cust_data['ScoreGlobale'] > 6, 'Segment'] = 'High-Value'

In [ ]:
cust_data.head()

,ClientID,cust,Récence,RécenceCluster,Fréquence,FréquenceCluster,Revenue,RevenueCluster,ScoreGlobale,Segment
0,13085.0,9999.0,57,3,92,3,1459.46,0,6,Mid-Value
1,18087.0,46.0,44,3,95,3,14411.62,0,6,Mid-Value
2,17519.0,116.0,33,3,224,3,5102.80,0,6,Mid-Value
3,12362.0,40.0,12,3,275,3,5284.58,0,6,Mid-Value
4,15712.0,38.0,9,3,167,3,3467.46,0,6,Mid-Value


Un graphique de dispersion (nuage de points) de Revenu en fonction de la Fréquence

In [ ]:

cust_graph = cust_data.query("Revenue < 50000 and Fréquence < 2000")


plot_data = [
    go.Scatter(
        x=cust_graph.query("Segment == 'Low-Value'")['Fréquence'],
        y=cust_graph.query("Segment == 'Low-Value'")['Revenue'],
        mode='markers',
        name='Low-Value',
        marker=dict(
            size=7,
            line=dict(width=1),
            color='blue',
            opacity=0.8
           )
    ),

    go.Scatter(
        x=cust_graph.query("Segment == 'Mid-Value'")['Fréquence'],
        y=cust_graph.query("Segment == 'Mid-Value'")['Revenue'],
        mode='markers',
        name='Mid-Value',
        marker=dict(
            size=9,
            line=dict(width=1),
            color='green',
            opacity=0.5
           )
    ),

    go.Scatter(
        x=cust_graph.query("Segment == 'High-Value'")['Fréquence'],
        y=cust_graph.query("Segment == 'High-Value'")['Revenue'],
        mode='markers',
        name='High-Value',
        marker=dict(
            size=11,
            line=dict(width=1),
            color='red',
            opacity=0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Revenue"},
        xaxis= {'title': "Fréquence"},
        title='Segments',
        title_x = 0.5,
        template="plotly_dark"
    )

fig = go.Figure(
        data=plot_data,
        layout=plot_layout
    )


fig.show(config={'displaylogo': False})

Graphique de dispersion (nuage de points) du Revenu en fonction de la Récence


In [ ]:
cust_graph = cust_data.query("Revenue < 50000 and Fréquence < 2000")

plot_data = [
    go.Scatter(
        x=cust_graph.query("Segment == 'Low-Value'")['Récence'],
        y=cust_graph.query("Segment == 'Low-Value'")['Revenue'],
        mode='markers',
        name='Low-Value',
        marker=dict(
            size=7,
            line=dict(width=1),
            color='blue',
            opacity=0.8
           )
    ),

    go.Scatter(
        x=cust_graph.query("Segment == 'Mid-Value'")['Récence'],
        y=cust_graph.query("Segment == 'Mid-Value'")['Revenue'],
        mode='markers',
        name='Mid-Value',
        marker=dict(
            size=9,
            line=dict(width=1),
            color='green',
            opacity=0.5
           )
    ),

    go.Scatter(
        x=cust_graph.query("Segment == 'High-Value'")['Récence'],
        y=cust_graph.query("Segment == 'High-Value'")['Revenue'],
        mode='markers',
        name='High-Value',
        marker=dict(
            size=11,
            line=dict(width=1),
            color='red',
            opacity=0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Revenue"},
        xaxis= {'title': "Récence"},
        title='Segments',
        title_x = 0.5,
        template="plotly_dark"
    )

fig = go.Figure(
        data=plot_data,
        layout=plot_layout
    )


fig.show(config={'displaylogo': False})

Graphique de dispersion (nuage de points) du Revenu en fonction de la Fréquence

In [ ]:
cust_graph = cust_data.query("Revenue < 50000 and Fréquence < 2000")

plot_data = [
    go.Scatter(
        x=cust_graph.query("Segment == 'Low-Value'")['Récence'],
        y=cust_graph.query("Segment == 'Low-Value'")['Fréquence'],
        mode='markers',
        name='Low-Value',
        marker= dict(
            size=7,
            line=dict(width=1),
            color='blue',
            opacity=0.8
           )
    ),

    go.Scatter(
        x=cust_graph.query("Segment == 'Mid-Value'")['Récence'],
        y=cust_graph.query("Segment == 'Mid-Value'")['Fréquence'],
        mode='markers',
        name='Mid-Value',
        marker=dict(
            size=9,
            line=dict(width=1),
            color='green',
            opacity=0.5
           )
    ),

    go.Scatter(
        x=cust_graph.query("Segment == 'High-Value'")['Récence'],
        y=cust_graph.query("Segment == 'High-Value'")['Fréquence'],
        mode='markers',
        name='High-Value',
        marker=dict(
            size=11,
            line=dict(width=1),
            color='red',
            opacity=0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Fréquence"},
        xaxis= {'title': "Récence"},
        title='Segments',
        title_x = 0.5,
        template="plotly_dark"
    )

fig = go.Figure(
        data=plot_data,
        layout=plot_layout
    )


fig.show(config={'displaylogo': False})

Créons une copie du dataframe cust_data et appliquons la méthode get_dummies pour convertir toutes les colonnes catégorielles, telles que Segment, en variables indicatrices.

In [ ]:
#Créer cust_class comme une copie de cust_data avant d'appliquer get_dummies
cust_class = cust_data.copy()
cust_class = pd.get_dummies(cust_class)
cust_class.head()


,ClientID,cust,Récence,RécenceCluster,Fréquence,FréquenceCluster,Revenue,RevenueCluster,ScoreGlobale,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value
0,13085.0,9999.0,57,3,92,3,1459.46,0,6,False,False,True
1,18087.0,46.0,44,3,95,3,14411.62,0,6,False,False,True
2,17519.0,116.0,33,3,224,3,5102.80,0,6,False,False,True
3,12362.0,40.0,12,3,275,3,5284.58,0,6,False,False,True
4,15712.0,38.0,9,3,167,3,3467.46,0,6,False,False,True


Étant donné que notre objectif est d'estimer si un client effectuera un achat au cours du prochain trimestre, nous allons créer une nouvelle colonne custRange avec des valeurs soit 1 soit 0 définies comme suit :
Si la valeur est 1, cela indique que le client achètera quelque chose au cours du prochain trimestre, c'est-à-dire dans les 90 jours suivant son dernier achat.
La valeur 0 indique que le client achètera quelque chose dans plus de 90 jours à partir de son dernier achat.

In [ ]:
cust_class['custRange'] = 1  ## inferieur a 3 mois
cust_class.loc[cust_class.cust>90,'custRange'] = 0 # plus de 3 months
cust_class.head()

,ClientID,cust,Récence,RécenceCluster,Fréquence,FréquenceCluster,Revenue,RevenueCluster,ScoreGlobale,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value,custRange
0,13085.0,9999.0,57,3,92,3,1459.46,0,6,False,False,True,0
1,18087.0,46.0,44,3,95,3,14411.62,0,6,False,False,True,1
2,17519.0,116.0,33,3,224,3,5102.80,0,6,False,False,True,0
3,12362.0,40.0,12,3,275,3,5284.58,0,6,False,False,True,1
4,15712.0,38.0,9,3,167,3,3467.46,0,6,False,False,True,1


Enfin, dans cette section, voyons la corrélation entre nos caractéristiques et notre variable cible. Nous pouvons le faire en appliquant la méthode corr au dataframe cust_data.

In [ ]:
corr_matrix = cust_class[cust_class.columns].corr()
corr_df = pd.DataFrame(corr_matrix.min())
corr_df.columns = ['MinCorrelationCoeff']
corr_df['MaxCorrelationCoeff'] = corr_matrix[corr_matrix < 1].max()
corr_matrix

,ClientID,cust,Récence,RécenceCluster,Fréquence,FréquenceCluster,Revenue,RevenueCluster,ScoreGlobale,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value,custRange
ClientID,1.000000,0.013109,0.024796,-0.027706,0.001097,-0.005765,-0.008548,-0.005408,-0.030648,0.020375,0.030444,-0.032822,-0.023490
cust,0.013109,1.000000,0.448860,-0.441134,-0.269017,0.270536,-0.163142,-0.103004,-0.392213,-0.069840,0.364701,-0.356049,-0.569324
Récence,0.024796,0.448860,1.000000,-0.964643,-0.251112,0.254100,-0.156418,-0.098179,-0.932764,-0.064973,0.857271,-0.848670,-0.503896
RécenceCluster,-0.027706,-0.441134,-0.964643,1.000000,0.247283,-0.254496,0.153447,0.094721,0.968468,0.063277,-0.892001,0.883565,0.517921
Fréquence,0.001097,-0.269017,-0.251112,0.247283,1.000000,-0.704083,0.475310,0.463662,0.121996,0.098423,-0.162250,0.150431,0.320559
FréquenceCluster,-0.005765,0.270536,0.254100,-0.254496,-0.704083,1.000000,-0.377565,-0.347102,-0.034674,-0.085431,0.143641,-0.133379,-0.363394
Revenue,-0.008548,-0.163142,-0.156418,0.153447,0.475310,-0.377565,1.000000,0.834119,0.160930,0.582501,-0.119967,0.050911,0.229213
RevenueCluster,-0.005408,-0.103004,-0.098179,0.094721,0.463662,-0.347102,0.834119,1.000000,0.129959,0.672203,-0.073807,-0.005814,0.182295
ScoreGlobale,-0.030648,-0.392213,-0.932764,0.968468,0.121996,-0.034674,0.160930,0.129959,1.000000,0.126915,-0.885049,0.869089,0.455888
Segment_High-Value,0.020375,-0.069840,-0.064973,0.063277,0.098423,-0.085431,0.582501,0.672203,0.126915,1.000000,-0.050233,-0.068152,0.122175


In [ ]:
corr_df

,MinCorrelationCoeff,MaxCorrelationCoeff
ClientID,-0.032822,0.030444
cust,-0.569324,0.448860
Récence,-0.964643,0.857271
RécenceCluster,-0.964643,0.968468
Fréquence,-0.704083,0.475310
FréquenceCluster,-0.704083,0.270536
Revenue,-0.377565,0.834119
RevenueCluster,-0.347102,0.834119
ScoreGlobale,-0.932764,0.968468
Segment_High-Value,-0.085431,0.672203


À partir de la sortie ci-dessus, nous observons que le score global présente la corrélation positive la plus élevée de 0,97 avec RécenceCluster, et Segment_Low-Value présente la corrélation négative la plus élevée de -0,99 avec Segment_Mid-Value.

Nous pouvons obtenir une bonne visualisation de la matrice des coefficients ci-dessous.

In [ ]:
plt.figure(figsize = (40, 30))
sns.heatmap(corr_matrix, annot = True, linewidths=0.2, fmt=".2f");

## Creation des models de machine learning

In [ ]:
cust_class = cust_class.drop('cust', axis=1)

In [ ]:
X, y = cust_class.drop('custRange', axis=1), cust_class.custRange
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None, shuffle=True)

In [ ]:
# Créer un tableau des Modéles de machine learning
models = []
models.append(("LogisticRegression", LogisticRegression()))
models.append(("GaussianNB", GaussianNB()))
models.append(("RandomForestClassifier", RandomForestClassifier()))
models.append(("SVC", SVC()))
models.append(("DecisionTreeClassifier", DecisionTreeClassifier()))
models.append(("xgb.XGBClassifier", xgb.XGBClassifier(eval_metric='mlogloss')))
models.append(("KNeighborsClassifier", KNeighborsClassifier()))

In [ ]:
# on va mesurer les metriques des diffrent models

scorer = MultiScorer({'accuracy'  : (accuracy_score , {}),
                      'f1_score'  : (f1_score       , {'pos_label': 3, 'average':'macro'}),
                      'recall'    : (recall_score   , {'pos_label': 3, 'average':'macro'}),
                      'precision' : (precision_score, {'pos_label': 3, 'average':'macro'})
                     })

# Un dictionnaire pour tous les modèles distincts et leurs métriques respectives
model_scores_dict = {'model_name' : [],
                     'accuracy'   : [],
                     'f1_score'   : [],
                     'recall'     : [],
                     'precision'  : [],
                     'time'       : []
                    }


# Pour chaque nom de modèle et modèle dans les modèles
for model_name, model in models:


    #ajouter model_name vers model_scores_dict
    model_scores_dict['model_name'].append(model_name)
    #print(model_name)
    kfold = KFold(n_splits=2, random_state=24, shuffle=True)
    start = time.time()
    _ = cross_val_score(model, X_train, y_train, cv = kfold, scoring = scorer)
    cv_result = scorer.get_results()


    for metric_name in cv_result.keys():
        # calcule le moyen de cv_result[metric_name]
        average_score = np.average(cv_result[metric_name])
        # mise a jour model_scores_dict avec average_score pour model_name
        model_scores_dict[metric_name].append(average_score)
        #print('%s : %f' %(metric_name, average_score))

    model_scores_dict['time'].append((time.time() - start))
    #print('time : ', time.time() - start, '\n\n')

model_score_df = pd.DataFrame(model_scores_dict).set_index("model_name")
model_score_df.sort_values(by=["accuracy", "f1_score", "time"], ascending=False)

,accuracy,f1_score,recall,precision,time
model_name,,,,,
LogisticRegression,0.904080,0.846486,0.849394,0.844100,0.158831
RandomForestClassifier,0.897638,0.845260,0.869589,0.833309,1.009089
GaussianNB,0.892867,0.842202,0.877290,0.825388,0.092745
xgb.XGBClassifier,0.885390,0.806123,0.814742,0.822320,0.243895
SVC,0.884097,0.798401,0.809267,0.824538,0.579504
DecisionTreeClassifier,0.882892,0.799913,0.809452,0.820043,0.038887
KNeighborsClassifier,0.879265,0.794919,0.800248,0.811614,0.302838


D'après l'ensemble de données, voici un résumé des observations :

La boutique en ligne compte 5942 clients provenant de 40 pays différents. Les trois principaux pays sont l'Allemagne, l'Espagne et la France, représentant 92 % des clients de Besins.

Les meilleures ventes de la boutique Besins ont eu lieu en novembre 2022, suivies de novembre 2023. De plus, il y a eu une augmentation des ventes mensuelles après le mois d'août.

La majorité des revenus de la boutique en ligne provient des clients résidant au Royaume-Uni, suivis de la République d'Irlande, des Pays-Bas, de l'Allemagne et de la France.

Enfin, de nouvelles fonctionnalités ont été introduites dans le dataframe cust_data, et un modèle d'apprentissage automatique non supervisé a été construit pour aider à décider si un client achètera à nouveau quelque chose dans la boutique en ligne au cours du prochain trimestre, c'est-à-dire 90 jours après la date de son dernier achat. Parmi les modèles utilisés, le modèle de classificateur XGB a dû être amélioré grâce au processus d'optimisation des hyperparamètres afin de surpasser le modèle de régression logistique.

